In [42]:
import wandb
import pandas as pd
import numpy as np
import os

In [35]:

entity = "evangeorgerex"
project = "fwal"
sweep_ids = [
    # '4p4opie9', # hyperparam tune mice_protein & madelon. F-Act no dropout
    # "7i2o46ri", # hyperparam tune all datasets. F-Act no dropout
    # 'bga4xu49', # hyperparam tune all datasets. F-Act
    # 'uw1zvim4', # F-Act finished PBMC
    'hfqtys2c', # mice_protein and madelon no dropout more patience

    
    # Baselines
    
    # 's5m2sqsv', # hyperparam tune all datasets. MLP
    # 'yr11wqmg', # hyperparam tune all datasets. XGBoost
    # 'rdvu3fg8', # hyperparam tune all datasets (except finance). LASSO  
    # 'xpxe64ee', # hyperparam tune finance. LASSO  
    # 'n6zb5lc9', # hyperparam tune all datasets SEFS
    
    # "22eckaxx", # rf baseline
    # "gty91unf", # rf baseline
    # "8yghctpp", # supervised cae
    # "h245pc2i", # cae
    
    # # finance:
    # "xclyvl0c", # supervised cae finance
    # "4upk4xrk", # RF finance
    # "jqulrkke", # cae finance
    
    # # PBMC
    # "vle39arl", # supervised cae
    # 'xh18ha0v', # CAE
    # "l102r3et", # rf
]

In [36]:

# entity = "evangeorgerex"
# project = "fwal"
# # 9npryg91", "7i2o46ri
# sweep_ids = [
#     # # FWAL models
#     # "ho0m3j5w", # FWAL
#     # # "iw8hr7oj", # FWAl hierarchical
#     # # "ryhck7mh", # FWAl hierarchical sigmoid 
#     # # "bwhcxdf3", # FWAL hierarchical shared
#     # # "54j0jkpy", # FWAL hierarchical shared sigmoid loss
#     # # "9npryg91", # FWAL hierarchical TTI loss
#     # "x9s9n05b", # FWAL hierarchical TTI loss L1 ablation
#     '4p4opie9', # hyperparam tune mice_protein & madelon. F-Act no dropout
#     "7i2o46ri", # hyperparam tune all datasets. F-Act no dropout
#     'bga4xu49', # hyperparam tune all datasets. F-Act
#     'uw1zvim4', # F-Act finished PBMC
#     'hfqtys2c', # mice_protein and madelon no dropout more patience

    
#     # Baselines
    
#     's5m2sqsv', # hyperparam tune all datasets. MLP
#     'yr11wqmg', # hyperparam tune all datasets. XGBoost
#     'rdvu3fg8', # hyperparam tune all datasets (except finance). LASSO  
#     'xpxe64ee', # hyperparam tune finance. LASSO  
#     'n6zb5lc9', # hyperparam tune all datasets SEFS
    
#     "22eckaxx", # rf baseline
#     "gty91unf", # rf baseline
#     "8yghctpp", # supervised cae
#     "h245pc2i", # cae
    
#     # finance:
#     "xclyvl0c", # supervised cae finance
#     "4upk4xrk", # RF finance
#     "jqulrkke", # cae finance
    
#     # PBMC
#     "vle39arl", # supervised cae
#     'xh18ha0v', # CAE
#     "l102r3et", # rf
#     # "iq7hs7sp", # SEFS PBMC - this broken (no finished runs)
#     # "a600kww9", # cae PBMC - this broken (no finished runs)
    
#     # old sweeps that are fine, but we have replaced with new hyperparam sweeps:
#     # "2bqm9fb6", # MLP finance - MLP results are here now: s5m2sqsv
#     # "tz0cpiih", # MLP PBMC - MLP results are here now: s5m2sqsv
#     # "40yayf8d", # MLP all - MLP results are here now: s5m2sqsv
#     # "u0xyvsw9", # xgboost all - XGB results are here now: yr11wqmg
#     # "4yfu2hrj", # XGBoost PBMC - XGB results are here now: yr11wqmg
#     # "18c8ij60", # xgboost finance - XGB results are here now: yr11wqmg
#     # "91aly80j", # FWAL finance (not sure which FWAL this is) - F-Act results are here now: bga4xu49
#     # "zkvbu18r", # lasso baseline - don't need this already covered by 'xpxe64ee' & 'rdvu3fg8'
#     # "efjov5qc", # lasso baseline - don't need this already covered by 'xpxe64ee' & 'rdvu3fg8'
#     # "0lh1b7hs", # SEFS - dont need this already covered by 'n6zb5lc9'
#     # "2xn215vh", # SEFS finance  - dont need this already covered by 'n6zb5lc9'
# ]

# # These sweeps are all broken because the reported bestmodel scores are wrong:
#     # "ycv2gbna", # fwal only reconstruct masked mice protein more patience
#     # "9tjs7cai", # fwal as mlp baseline
#     # "nmsvf0u4", # fwal new datasets
#     # "5kxy8gl8", # fwal as mlp baseline
#     # "t1z9hleb", # fwal tti 
#     # "3ubxo03v", # finish fwal tti
#     # "t8i7yh69", # fwal gisette
#     # "cwhd81fy", # fwal MNIST (states are failed despite being finished for some reason)
#     # "z9b6sox8", # fwal only reconstruct masked
#     # "xi4jr64q", # fwals only reconstruct masked GPU
#     # "aie2q1ph", # cae
#     # "uunpksug", # cae

In [ ]:
def get_tti_results(run):
    '''
    returns:
        float: optimal test-time intervention validation f1
        float: optimal test-time intervention test f1
        float: full test-time intervention validation f1
        float: full test-time intervention test f1
    '''

    f1_weighted = []
    num_features = []

    history_dict = {}

    # Iterate through the run history to extract the desired metrics
    keys = [
        "tti_valid_metrics.F1_weighted", "tti_test_metrics.F1_weighted", "num_additional_features"
        ]
    for row in run.scan_history(keys=keys):
        # print('.')
        # Check if both keys exist in the current row
        for key in keys:
            if key in row and row[key] is not None:
                if key not in history_dict:
                    history_dict[key] = []
                history_dict[key].append(row[key])
            else:
                # print(key)
                continue
            
    max_index = np.argmax(history_dict['tti_valid_metrics.F1_weighted'])
    opt_tti_valid = history_dict['tti_valid_metrics.F1_weighted'][max_index]
    opt_tti_test = history_dict['tti_test_metrics.F1_weighted'][max_index]
    
    full_tti_valid = history_dict['tti_valid_metrics.F1_weighted'][-1]
    full_tti_test = history_dict['tti_test_metrics.F1_weighted'][-1]
    
    return opt_tti_valid, opt_tti_test, full_tti_valid, full_tti_test 
    

def fetch_runs(entity, project, sweep_ids):
    api = wandb.Api()
    runs_data = []

    for sweep_id in sweep_ids:
        sweep_runs = api.sweep(f"{entity}/{project}/{sweep_id}").runs
        for run in sweep_runs:
            # Check if the run's state is 'finished'
            if run.state == "finished" or sweep_id == "cwhd81fy":
                dataset_name = run.config.get("dataset", "Unknown")
                if dataset_name in ["poly_binarised_decimalised_mod10_synth", "simple_trig_synth", "Unknown"]:
                    continue
                
                if run.config.get("as_MLP_baseline", False):
                    model_name = "MLP_same_capacity"
                else:
                    model_name = run.config.get("model", "Unknown")
                    
                    if model_name == "fwal":
                        model_name = "F-Act"
                        
                    # if run.config.get("hierarchical", False):
                    #     model_name = "F-Act hierarchical"
                    #     if run.config.get("share_mask", False):
                    #         model_name = "F-Act hierarchical shared"
                    #         if run.config.get("sigmoid_loss", False):
                    #             model_name = "F-Act hierarchical shared L_sig"
                    # if sweep_id == "x9s9n05b":
                    #     model_name = "F-Act hierarchical TTI (old)"
                    # if sweep_id in ["9npryg91", "7i2o46ri"]:
                    #     model_name = "F-Act hierarchical TTI"
                run_data = {
                    "model": model_name,  # Assuming model name is stored in 'model_name'
                    "dataset": dataset_name,  # Assuming dataset name is stored in 'dataset_name'
                    "seed": run.config.get("seed_model_init", None),
                    "bestmodel_train/F1_weighted": run.summary.get("bestmodel_train/F1_weighted", float('nan')),
                    "bestmodel_valid/F1_weighted": run.summary.get("bestmodel_valid/F1_weighted", float('nan')),
                    "bestmodel_test/F1_weighted": run.summary.get("bestmodel_test/F1_weighted", float('nan')),
                }
                
                
                # Specific hyperparameters
                if model_name == "lasso":
                    run_data["lasso_C"] = run.config.get("lasso_C")
                    run_data["lasso_l1_ratio"] = run.config.get("lasso_l1_ratio")
                elif model_name == "rf":
                    run_data["rf_max_depth"] = run.config.get("rf_max_depth")
                elif model_name in ["cae", "supervised_cae"]:
                    run_data["CAE_neurons_ratio"] = run.config.get("CAE_neurons_ratio")
                elif model_name == "F-Act hierarchical TTI":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "F-Act hierarchical TTI (old)":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "F-Act":
                    if sweep_id in ["bga4xu49"]:
                        run_data['dropout'] = 0.2
                        run_data["lr"] = run.config.get("lr")
                        run_data["num_hidden"] = run.config.get("num_hidden")
                        run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                    else:
                        # sweep_id == "7i2o46ri":
                        run_data['dropout'] = 0.0
                        run_data["lr"] = run.config.get("lr")
                        run_data["num_hidden"] = 4
                        run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "xgboost":
                    run_data["xgb_eta"] = run.config.get("xgb_eta")
                    run_data["xgb_max_depth"] = run.config.get("xgb_max_depth")
                elif model_name in ["MLP_same_capacity", "SEFS"]:
                    run_data["lr"] = run.config.get("lr")
                    run_data["num_hidden"] = run.config.get("num_hidden")
                    
                    
                    
                runs_data.append(run_data)
                if run_data['model'] == 'F-Act':
                    
                    opt_tti_valid, opt_tti_test, full_tti_valid, full_tti_test = get_tti_results(run)
                    opt_tti_run_data = run_data.copy()
                    opt_tti_run_data['bestmodel_valid/F1_weighted'] = opt_tti_valid
                    opt_tti_run_data['bestmodel_test/F1_weighted'] = opt_tti_test
                    opt_tti_run_data['model'] = 'F-Act (optimal TTI)'
                    runs_data.append(opt_tti_run_data)
                    
                    full_tti_run_data = run_data.copy()
                    full_tti_run_data['bestmodel_valid/F1_weighted'] = full_tti_valid
                    full_tti_run_data['bestmodel_test/F1_weighted'] = full_tti_test
                    full_tti_run_data['model'] = 'F-Act (full TTI)'
                    runs_data.append(full_tti_run_data)
                
    return runs_data

def process_runs_data(runs_data):
    df = pd.DataFrame(runs_data)
    
    # Models with hyperparameter sweeps
    models_with_hyperparams = [
        'lasso', 'rf', 'cae', 'supervised_cae', 'F-Act hierarchical TTI', 'F-Act hierarchical TTI (old)',
        'F-Act', 'MLP_same_capacity', 'SEFS', 'xgboost', 
    ]
    best_hyperparams = {}
    
    for model in models_with_hyperparams:
        model_df = df[df['model'] == model]

        # For the 'F-Act' model, first find the best seed for each combination of hyperparameters
        if model == 'F-Act':
            max_seed_df = model_df.groupby(['dataset', 'sparsity_regularizer_hyperparam', 'lr', 'num_hidden', 'dropout', 'seed'])['bestmodel_valid/F1_weighted'].idxmax()
            model_df = model_df.loc[max_seed_df]
        
        # Further aggregate to find the best hyperparameter combination per dataset
        group_columns = {
            'lasso': ['dataset', 'lasso_C', 'lasso_l1_ratio'],
            'rf': ['dataset', 'rf_max_depth'],
            'cae': ['dataset', 'CAE_neurons_ratio'],
            'supervised_cae': ['dataset', 'CAE_neurons_ratio'],
            'F-Act hierarchical TTI': ['dataset', 'sparsity_regularizer_hyperparam'],
            'F-Act hierarchical TTI (old)': ['dataset', 'sparsity_regularizer_hyperparam'],
            'F-Act': ['dataset', 'sparsity_regularizer_hyperparam', 'lr', 'num_hidden', 'dropout'],
            'xgboost': ['dataset', 'xgb_eta', 'xgb_max_depth'],
            'MLP_same_capacity': ['dataset', 'lr', 'num_hidden'],
            'SEFS': ['dataset', 'lr', 'num_hidden']
        }

        best_combinations = model_df.groupby(group_columns[model])['bestmodel_valid/F1_weighted'].mean().reset_index()
        # Identify the best hyperparameters for each dataset
        best_combinations = best_combinations.loc[best_combinations.groupby('dataset')['bestmodel_valid/F1_weighted'].idxmax()]
        best_hyperparams[model] = best_combinations
        
        # Create a tuple representation for easy comparison
        best_combinations_tuples = best_combinations[group_columns[model]].apply(tuple, axis=1)

        # Filter the original df to include only runs with the best hyperparameters for the given model
        df = df[~((df['model'] == model) & ~df[group_columns[model]].apply(tuple, axis=1).isin(best_combinations_tuples)) | (df['model'] != model)]
    
    # Now, group by model and dataset to average across seeds for the final analysis
    final_grouped = df.groupby(['model', 'dataset']).mean().reset_index()
    
    # Printing best hyperparameters for each model
    for model, params_df in best_hyperparams.items():
        print(f"Best hyperparameters for {model}:")
        print(params_df)
        print("\n")
    
    return final_grouped, best_hyperparams

# Fetch and process the data
runs_data = fetch_runs(entity, project, sweep_ids)
averaged_data, best_hyperparams = process_runs_data(runs_data)

In [ ]:
best_hyperparams.keys()
lasso_hyperparams = best_hyperparams['lasso']
rf_hyperparams = best_hyperparams['rf']
cae_hyperparams = best_hyperparams['cae']
supervised_cae_hyperparams = best_hyperparams['supervised_cae']
# Fact_hyperparams = best_hyperparams['F-Act hierarchical TTI']
xgb_hyperparams = best_hyperparams['xgboost']
Fact_hyperparams = best_hyperparams['F-Act'] 
MLP_hyperparams = best_hyperparams['MLP_same_capacity'] 
SEFS_hyperparams = best_hyperparams['SEFS'] 


In [ ]:
lasso_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("Lasso Best Hyperparameters")

,dataset,lasso_C,lasso_l1_ratio,bestmodel_valid/F1_weighted
2,COIL20,10.000000,0.500000,0.982551
21,Isolet,100.000000,0.250000,0.947942
30,PBMC,10.000000,0.000000,0.873614
48,USPS,10.000000,0.750000,0.934424
62,finance,10.000000,0.500000,0.595040
75,madelon,10.000000,0.000000,0.548075
95,mice_protein,100.000000,0.000000,0.964513
112,poly_binarised_decimalised_synth,100.000000,0.500000,0.502693


In [ ]:
rf_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("RF Best Hyperparameters")

,dataset,rf_max_depth,bestmodel_valid/F1_weighted
2,COIL20,7.000000,0.970085
5,Isolet,7.000000,0.921524
8,MNIST,7.000000,0.910721
11,PBMC,7.000000,0.869854
14,USPS,7.000000,0.940427
17,finance,7.000000,0.600702
20,gisette,7.000000,0.949999
23,madelon,7.000000,0.693069
26,mice_protein,7.000000,0.962906
29,poly_binarised_decimalised_synth,7.000000,0.506308


In [ ]:
cae_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("CAE Best Hyperparameters")

,dataset,CAE_neurons_ratio,bestmodel_valid/F1_weighted
9,COIL20,1.000000,0.983572
17,Isolet,0.800000,0.828027
20,PBMC,0.100000,0.667353
30,USPS,1.000000,0.904831
37,finance,0.700000,0.611487
50,madelon,1.000000,0.727068
56,mice_protein,0.600000,0.894743
68,poly_binarised_decimalised_synth,0.800000,0.419617


In [ ]:
supervised_cae_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("CAE Best Hyperparameters")

,dataset,CAE_neurons_ratio,bestmodel_valid/F1_weighted
0,COIL20,0.100000,0.058931
11,Isolet,0.200000,0.039760
20,PBMC,0.100000,0.832341
21,USPS,0.100000,0.212480
36,finance,0.600000,0.547139
50,madelon,1.000000,0.624305
60,mice_protein,1.000000,0.167532
63,poly_binarised_decimalised_synth,0.300000,0.368871


In [ ]:
xgb_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("XGBoost Best Hyperparameters")

,dataset,xgb_eta,xgb_max_depth,bestmodel_valid/F1_weighted
1,COIL20,0.100000,6.000000,0.985390
11,Isolet,0.100000,9.000000,0.902887
21,PBMC,0.300000,3.000000,0.898703
30,USPS,0.300000,3.000000,0.968251
43,finance,0.500000,6.000000,0.634798
46,madelon,0.100000,6.000000,0.807590
57,mice_protein,0.300000,3.000000,0.978342
65,poly_binarised_decimalised_synth,0.100000,9.000000,0.504693


In [ ]:
Fact_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("F-Act")

,dataset,sparsity_regularizer_hyperparam,lr,num_hidden,dropout,bestmodel_valid/F1_weighted
6,COIL20,0.000000,0.001000,1.000000,0.200000,0.992910
74,Isolet,0.100000,0.000300,1.000000,0.200000,0.926447
157,PBMC,10.000000,0.000100,4.000000,0.200000,0.846068
188,USPS,0.100000,0.001000,1.000000,0.200000,0.968084
246,finance,0.100000,0.000300,1.000000,0.200000,0.599168
319,madelon,0.100000,0.003000,4.000000,0.000000,0.728721
409,mice_protein,0.500000,0.001000,4.000000,0.000000,0.990736
485,poly_binarised_decimalised_synth,1.000000,0.000300,1.000000,0.200000,0.273722


In [ ]:
MLP_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("MLP")

,dataset,lr,num_hidden,bestmodel_valid/F1_weighted
6,COIL20,0.001000,1.000000,0.992998
16,Isolet,0.001000,2.000000,0.944965
26,PBMC,0.001000,4.000000,0.898915
33,USPS,0.001000,1.000000,0.969688
43,finance,0.001000,2.000000,0.570507
51,madelon,0.001000,1.000000,0.593281
54,mice_protein,0.000100,1.000000,0.992254
64,poly_binarised_decimalised_synth,0.000100,2.000000,0.503680


In [ ]:
SEFS_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("SEFS")

,dataset,lr,num_hidden,bestmodel_valid/F1_weighted
0,COIL20,0.003000,1.000000,0.963678
3,Isolet,0.003000,1.000000,0.882264
6,USPS,0.003000,1.000000,0.935731
10,finance,0.003000,2.000000,0.580251
13,madelon,0.003000,2.000000,0.642004
15,mice_protein,0.003000,1.000000,0.860212
18,poly_binarised_decimalised_synth,0.003000,1.000000,0.362826


In [ ]:
# Pivoting for 'bestmodel_train/F1_weighted'
train_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_train/F1_weighted')
# print("Training F1 Weighted Averages:")
# print(train_pivot)
# print("\n")  # Add some space between tables

# Pivoting for 'bestmodel_valid/F1_weighted'
valid_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_valid/F1_weighted')
# print("Validation F1 Weighted Averages:")
# print(valid_pivot)
# print("\n")  # Add some space between tables

# Pivoting for 'bestmodel_test/F1_weighted'
test_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_test/F1_weighted')
# print("Test F1 Weighted Averages:")
# print(test_pivot)
for df in [train_pivot, valid_pivot, test_pivot]:
    df.drop('MNIST', axis=1, inplace=True)
    df.drop('gisette', axis=1, inplace=True) 
    df.drop('poly_binarised_decimalised_synth', axis=1, inplace=True)

def highlight_max(s):
    '''
    Highlight the maximum in a Series by changing font color to red and making it bold.
    '''
    is_max = s == s.max()
    return ['color: green; font-weight: bold' if v else '' for v in is_max]


table_styles = [
    {'selector': 'th',
     'props': [('background', '#606060'), ('color', 'white')]
    }
]

In [ ]:
train_pivot.style.set_table_styles(table_styles).set_caption("Training F1 Weighted Averages").apply(highlight_max)

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,1.000000,0.999628,1.000000,0.990255,0.658565,0.869327,0.941378
MLP_same_capacity,1.000000,0.999255,0.999349,0.990137,0.610960,0.819199,0.999479
SEFS,0.979644,0.976174,nan,0.938292,0.615141,0.761723,0.904336
cae,1.000000,0.962531,0.882927,0.930945,0.656600,0.873945,0.962683
lasso,1.000000,1.000000,1.000000,0.994983,0.670358,0.803830,0.988156
rf,0.986327,0.984742,0.937396,0.957755,0.877761,0.995769,0.999807
supervised_cae,0.054281,0.040158,0.830378,0.207287,0.537526,0.610881,0.191261
xgboost,1.000000,1.000000,0.995177,1.000000,0.861745,1.000000,1.000000


In [ ]:
valid_df = valid_pivot.style.set_table_styles(table_styles).set_caption("Valid F1 Weighted Averages").apply(highlight_max)
valid_df

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.992910,0.926447,0.846068,0.968084,0.599168,0.728721,0.904343
MLP_same_capacity,0.992998,0.944965,0.898915,0.969688,0.570507,0.593281,0.992254
SEFS,0.963678,0.882264,nan,0.935731,0.580251,0.642004,0.860212
cae,0.983572,0.828027,0.667353,0.904831,0.611487,0.727068,0.894743
lasso,0.982551,0.947942,0.873614,0.934424,0.595040,0.548075,0.964513
rf,0.970085,0.921524,0.869854,0.940427,0.600702,0.693069,0.962906
supervised_cae,0.058931,0.039760,0.832341,0.212480,0.547139,0.624305,0.167532
xgboost,0.985390,0.902887,0.898703,0.968251,0.634798,0.807590,0.978342


In [ ]:
test_df = test_pivot.style.set_table_styles(table_styles).set_caption("Test F1 Weighted Averages").apply(highlight_max)
test_df

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.986039,0.919675,0.831719,0.968292,0.602923,0.722480,0.903048
MLP_same_capacity,0.988311,0.934809,0.894191,0.967768,0.570214,0.571757,0.983014
SEFS,0.949701,0.886111,nan,0.925440,0.599271,0.652284,0.850778
cae,0.970427,0.801414,0.680679,0.904749,0.592564,0.701883,0.853472
lasso,0.982413,0.945832,0.892459,0.933578,0.597777,0.515320,0.952401
rf,0.967561,0.900926,0.886592,0.933634,0.619456,0.671901,0.969803
supervised_cae,0.064637,0.036833,0.853663,0.209018,0.544434,0.618427,0.172394
xgboost,0.986075,0.887528,0.894172,0.973689,0.588311,0.809609,0.981490


In [34]:
def to_latex(df):
    latex_table = df.to_latex()
    modified_string = latex_table.replace(r"\th#606060", r"\centering")
    modified_string = modified_string.replace(r"MLP_same_capacity", r"MLP")
    # \n\\begin{tabular}
    modified_string = modified_string.replace("\n\\begin{tabular}{lrrrrrrr}", "\n\\begin{tabular}{lrrrrrrr}\n\\hline")
    modified_string = modified_string.replace("\\\\\nmodel &  &  &  &  &  &  &  \\\\\nF-Act", "\\\\\nmodel &  &  &  &  &  &  &  \\\\\n\\hline\nF-Act")
    modified_string = modified_string.replace("\n\\end{tabular}", "\n\\hline\n\\end{tabular}")


    import re

    def replace_color_and_bold(text):
        # Define a regex pattern to match the specific format
        pattern = r'\\colorgreen \\font-weightbold (\d+\.\d+)'

        # Define a function to replace each match
        def replace_match(match):
            number = match.group(1)  # This is the number captured by (\d+\.\d+)
            return r'\textcolor{green}{\textbf{' + number + '}}'

        # Use re.sub to replace all occurrences in the text
        modified_text = re.sub(pattern, replace_match, text)
        return modified_text

    # Example usage
    modified_string = replace_color_and_bold(modified_string)

    print(modified_string)

to_latex(test_df)

\begin{table}
\caption{Test F1 Weighted Averages}
\centering
\begin{tabular}{lrrrrrrr}
\hline
dataset & COIL20 & Isolet & PBMC & USPS & finance & madelon & mice_protein \\
model &  &  &  &  &  &  &  \\
\hline
F-Act & 0.986039 & 0.919675 & 0.831719 & 0.968292 & 0.602923 & 0.722480 & 0.903048 \\
MLP & \textcolor{green}{\textbf{0.988311}} & 0.934809 & \textcolor{green}{\textbf{0.894191}} & 0.967768 & 0.570214 & 0.571757 & \textcolor{green}{\textbf{0.983014}} \\
SEFS & 0.949701 & 0.886111 & nan & 0.925440 & 0.599271 & 0.652284 & 0.850778 \\
cae & 0.970427 & 0.801414 & 0.680679 & 0.904749 & 0.592564 & 0.701883 & 0.853472 \\
lasso & 0.982413 & \textcolor{green}{\textbf{0.945832}} & 0.892459 & 0.933578 & 0.597777 & 0.515320 & 0.952401 \\
rf & 0.967561 & 0.900926 & 0.886592 & 0.933634 & \textcolor{green}{\textbf{0.619456}} & 0.671901 & 0.969803 \\
supervised_cae & 0.064637 & 0.036833 & 0.853663 & 0.209018 & 0.544434 & 0.618427 & 0.172394 \\
xgboost & 0.986075 & 0.887528 & 0.894172 & \textcolor